In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from pathlib import Path as path
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings("ignore")

# Directory containing the dataset
data_dir = 'C:/Users/Asus/Downloads/Tensor/New folder/seg_train/seg_train'
save_dir = 'C:/Users/Asus/Downloads/Tensor/New folder/Predicted_Images'  # Directory to save correctly predicted images

# Define parameters for image size and batch size
img_height = 180
img_width = 180
batch_size = 32

# Use ImageDataGenerator to load images in batches and split dataset
datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    validation_split=0.2      # Reserve 20% of the data for validation
)

# Load training data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=None  # Set seed to None for better randomness
)


""""
data_dir: The path to the directory where the images are stored.

target_size=(img_height, img_width): The images are resized to the dimensions defined earlier, i.e., 180x180 pixels.

batch_size=batch_size: Specifies the number of images to be processed in each batch. Here, it’s set to 32, so the model will process 32 images in each iteration.

class_mode='sparse': This means the labels are encoded as integers instead of one-hot encoding. This is useful when you have multi-class classification, and the labels are in integer form, making it more memory-efficient. Each image will be assigned an integer representing its class.

subset='training': Refers to the 80% portion of the dataset that is used for training, as set by the validation_split=0.2 earlier in the ImageDataGenerator.

shuffle=True: Ensures that the data is shuffled after every epoch. This helps prevent the model from learning in sequence, which could lead to overfitting.

seed=None: No specific seed is set for random number generation. This allows for better randomness when shuffling the dataset during training.
"""



# Load validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='sparse',
    subset='validation',
    shuffle=True,
    seed=None  # Set seed to None for better randomness
)

"""batch_size=1: For validation, 
the batch size is set to 1. 
This means each image is evaluated individually. 
Validation typically doesn’t require large batch sizes since we are not training the model but only evaluating it.
"""



# Define class names
class_names = {
    0: 'building',
    1: 'forest',
    2: 'glacier',
    3: 'mountain',
    4: 'sea',
    5: 'street'
}

"""
This is a dictionary that maps the integer class labels (0, 1, 2, 3, 4, 5) to their corresponding category names.
"""


# Retrieve a truly random batch of images and labels
X_train_scaled, y_train = next(train_generator)  # Get the next batch of images and labels


"""
The train_generator is an iterator that loads batches of images and labels on the fly. This line retrieves the next batch of images and corresponding labels.
X_train_scaled: This variable stores the batch of images that have been loaded and preprocessed (rescaled and resized). Each image has been normalized (pixel values scaled to [0, 1]).
y_train: This contains the labels (class IDs) associated with the images in the batch.
"""


# Randomly select an index from the batch
index_to_display = np.random.randint(len(X_train_scaled))
random_image = X_train_scaled[index_to_display]

"""

This generates a random integer between 0 and the length of X_train_scaled (which equals the batch size, i.e., 32). This random integer represents the index of a randomly chosen image within the batch.
For example, if the batch has 32 images, this function could generate a random number from 0 to 31, allowing you to randomly select an image.

Once the random index is selected, this line extracts the corresponding image (random_image) from the batch X_train_scaled.
This image will be used for visualization or further processing.

"""


# Display the random image
plt.imshow(random_image)
plt.title(f'Class: {class_names[int(y_train[index_to_display])]}')
plt.axis('off')
plt.show()

"""
The title for the displayed image is set using the class name.

y_train[index_to_display]: This retrieves the class label (an integer) of the random_image from the corresponding y_train array.

int(y_train[index_to_display]): Converts the label to an integer, as y_train might store the labels in float format due to preprocessing.

class_names[...]: This dictionary is used to convert the integer label into a human-readable class name (e.g., 'forest', 'sea', etc.).

The f'...' format string ensures the class name is dynamically added to the title.

plt.axis('off'):

This removes the axis around the image (x and y coordinates), so only the image is displayed without any borders, ticks, or labels.

"""



# Print the scaled pixel values of the image
print(f'Scaled pixel values of the image')
print(random_image)



num_classes = 6

# CNN model building
model = tf.keras.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D(2, 2),  # Pooling layer
    layers.Dropout(0.25),  # Dropout layer
    
    
"""
    Conv2D Layer: This is a 2D convolutional layer that learns filters/kernels, which are applied to the input images.
filters=32: The layer uses 32 filters, meaning it will produce 32 different feature maps. 
Filters help detect features like edges, textures, and patterns.(The numbers 32, 64, and 128 are Powers of two are commonly used because they are efficient for computation on digital hardware, such as GPUs, due to their binary nature. This choice makes memory allocation, parallelism, and processing faster and more efficient.)
kernel_size=(3, 3): This is the size of each filter. A 3x3 filter moves over the image and learns spatial patterns.
activation='relu': The ReLU (Rectified Linear Unit) activation function is applied to the output of the convolution to introduce non-linearity. 
It replaces negative values with 0, keeping only positive values.

MaxPooling2D(2, 2): This is a pooling layer that reduces the spatial dimensions (height and width) of the feature maps.
It takes a 2x2 block of pixels and selects the maximum value, hence the name "max pooling."
Purpose: This layer down-samples the feature maps, reducing computational load, preventing overfitting, and retaining only the most important information.


Dropout(0.25): Dropout is a regularization technique that randomly drops (sets to zero) 25% of the neurons in the layer during each training step.
Purpose: Prevents overfitting by ensuring that the model does not rely too heavily on specific neurons and instead learns more generalized features.


"""    
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D(2, 2),  # Pooling layer
    layers.Dropout(0.25),  # Dropout layer

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D(2, 2),  # Pooling layer
    layers.Dropout(0.25),  # Dropout layer

    layers.Flatten(),  # Flatten layer
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Dropout layer
    layers.Dense(num_classes, activation='softmax')  # Output layer
])

"""
Flatten(): Converts the 2D feature maps output from the last pooling layer into a 1D vector. 
This is necessary because fully connected layers (Dense layers) expect 1D input.

"""


model.summary() # summary of the model

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(
    train_generator,  # Training data generator
    epochs=20,  # Number of epochs to train the model
    validation_data=validation_generator,  # Validation data generator
    steps_per_epoch=train_generator.samples // batch_size,  # Number of batches per epoch
    validation_steps=validation_generator.samples // batch_size  # Number of batches in validation
)


# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


# Make predictions on a random batch from the validation set
X_val_scaled, y_val = next(validation_generator)  # Get a batch of validation data

# Predict the classes for the batch
predictions = model.predict(X_val_scaled)

# Randomly select an index from the batch for visualization
index_to_display = np.random.randint(len(X_val_scaled))

# Get the predicted class for the selected sample
predicted_class = np.argmax(predictions[index_to_display])

# Get the true class for the selected sample
true_class = int(y_val[index_to_display])  # Convert to integer if needed

# Display the selected image with the predicted and true classes
plt.imshow(X_val_scaled[index_to_display])
plt.title(f'Predicted Class: {class_names[predicted_class]}, True Class: {class_names[true_class]}')
plt.axis('off')  # Hide the axes
plt.show()


# Print the result of the prediction
if predicted_class == true_class:
    print(f'Correct prediction! Predicted: {class_names[predicted_class]}, True: {class_names[true_class]}')
else:
    print(f'Incorrect prediction. Predicted: {class_names[predicted_class]}, True: {class_names[true_class]}')

In [ ]:
# final step to evaluate the model on the test data and save the predicted images along with their classified labels in the appropriate directories
test_dir = 'C:/Users/Asus/Downloads/Tensor/New folder/seg_test/'
save_dir = 'C:/Users/Asus/Downloads/Tensor/New folder/Predicted_Images'  # Directory to save correctly predicted images

# Create directories for each class inside the save_dir
for class_name in class_names.values():
    os.makedirs(os.path.join(save_dir, class_name), exist_ok=True)

# Use ImageDataGenerator for the test set (without validation split)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='sparse',
    shuffle=False  # No shuffling so that results are consistent with file order
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Predict on the test set
test_generator.reset()  # Reset the generator to avoid any side effects
predictions = model.predict(test_generator)

# Iterate over each image in the test set
for i in range(len(predictions)):
    predicted_class = np.argmax(predictions[i])  # Get predicted class index
    true_class = int(test_generator.labels[i])  # Get true class index

    # Get the file path of the current image
    img_path = test_generator.filepaths[i]
    img_name = os.path.basename(img_path)

    # Load the image (optional, for display or saving later)
    img = PIL.Image.open(img_path)

    # Print prediction result
    if predicted_class == true_class:
        print(f'Correctly predicted: {class_names[predicted_class]} for image {img_name}')
    else:
        print(f'Incorrectly predicted: {class_names[predicted_class]} (True: {class_names[true_class]}) for image {img_name}')

    # Save the image to the predicted class directory
    save_path = os.path.join(save_dir, class_names[predicted_class], img_name)
    img.save(save_path)

print(f'All predicted images saved to {save_dir}.')
